# Simulations of SARS-CoV-2 neutralizing Ab sensors

Basile Wicky – last updated 210518

1) Ternary: system composed of ACE2, RBD and nAb (e.g. [previously published](https://www.sciencedirect.com/science/article/pii/S0956566321001597?casa_token=ziGglpWaZaoAAAAA:QOrrHf75nx7H20VbNVZaDvmpEFAcQcOtefpNhV9vC6yGu6zGwWI_xKvMekJN4tdxhHf5SnbSY_s#fig5)).

2) Quaternary: system composed of Cage-Latch, Key, RBD and nAb (this work).

*NB* some regions of parameter phase-space are numerically unstable and can lead to the simulation crashing.

---
# Jump to interactive plotting

[Ternary system](#ternary_system)

[Quaternary system](#quaternary_system)

In [1]:
import numpy as np
import seaborn as sns; sns.set_context('talk')
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import interact, widgets # for interactive plotting
from scipy.integrate import odeint

# Ternary system
---

> __Species:__\
ACE2\
RBD\
nAb\
\
__Equilibria:__\
ACE2 + RBD <=> ACE2:RBD\
RBD + nAb <=> RBD:nAb

- ACE2 and RBD can interact.

- RBD and nAb can interact.

- The binary complex ACE2:RBD gives out the signal.

In [2]:
def ODEsystem_ternary(conc, t, ks): # system of ODEs to be solved by odeint()
    '''
    Returns a system of ODEs for numerical integration
    ---
    t: time-step of the ODE integration
    Conc: tuple of concentrations; unknowns
    k: tuple of kinetic constants; constants of the system
    ---
    A = ACE2
    B = RBD
    C = nAb
    Affinity of A:B complex is defined by K1 = k_1 / k1 
    Affinity of B:C complex is defined by K2 = k_2 / k2
    '''
    
    A, B, C, AB, BC = conc    
    k1, k_1, k2, k_2 = ks
    
    # coupled ODEs describing the system
    dA_dt  = - k1*A*B + k_1*AB
    dB_dt  = - k1*A*B + k_1*AB - k2*B*C + k_2*BC
    dC_dt  = - k2*B*C + k_2*BC
    dAB_dt = k1*A*B - k_1*AB
    dBC_dt = k2*B*C - k_2*BC
    
    return [
        dA_dt,
        dB_dt,
        dC_dt,
        dAB_dt,
        dBC_dt
    ]

def solve_ternary(K1, K2, ACE2, RBD, nAb):
    '''
    Solves for the equililibrum of the system as a function of affinities and concentrations
    Returns the the time-evolution of the fraction of each species
    --
    res: number of nAb concentration points that are calculated (affects calculation speed)
    K1: equilibrium dissociation constant between ACE2 and RBD (in M)
    K2: equilibrium dissociation constant between RBD and nAb (in M)
    ACE2: total concentration of ACE2 (in M)
    RBD: total concentration of RBD (in M)
    '''
    
    k1, k2 = 1e6, 1e6 # arbitrary association rate constants (in M^-1 s^-1)
    # although the model is based on ODEs (kinetics), only consider steady-state of the system
    # thus the actual kinetic rates are unimportant, as only the ratios (equilibrium) matter
    
    # get dissociation rate constants from association rate constants and equilibrium dissociation constants
    k_1 = k1 * K1
    k_2 = k2 * K2
    
    ks = k1, k_1, k2, k_2

    # inital concentrations used by ODE solver (in M)
    A0 = ACE2
    B0 = RBD 
    C0 = nAb
    AB0 = 0
    BC0 = 0        
    
    init = [A0, B0, C0, AB0, BC0]
    
    frac = []
    
    # perform integrations over 1s first, then increase if system has not reached equilibrium
    sections = 0
    maxcounter = 0
    reached_eq = [False] * len(init)
    
    while not all(reached_eq) and maxcounter < 50: # adaptive integration until steady-state is reached

        t = np.linspace(0, 10**sections, 10000)

        # solve the system of ODEs
        sol = odeint(
            ODEsystem_ternary, 
            init, 
            t,
            args=(ks,),
            mxstep=5000           
        )
    
        # compute fraction for each species
        frac.append(sol[:, 0] / ACE2)
        frac.append(sol[:, 1] / RBD)
        frac.append(sol[:, 2] / nAb)
        frac.append(sol[:, 3] / min(ACE2, RBD))
        frac.append(sol[:, 4] / min(RBD, nAb)) 
    
        # test if system reached equilibrium by computing the gradient
        gradients = np.array(np.gradient(np.array(frac), axis=-1))
        reached_eq = [True if abs(endpoint) < 1e-7 else False for endpoint in np.mean(gradients[:, -100:], axis=-1)]

        if not all(reached_eq) and maxcounter < 50: # check if system reached equilibrium
            frac = [] # erase
            sections += 1  
        
        maxcounter += 1

    return np.array(frac)

def solve_ternary_Lmax(K1, ACE2, RBD):
    '''
    Solves quadratic system for the equilibrium between ACE2 and RBD
    Returns the fraction of complex
    '''
    
    A0 = ACE2
    R0 = RBD
    
    AR = ((A0 + R0 + K1) - np.sqrt((A0 + R0 + K1)**2 - 4 * A0 * R0)) / 2
    
    fAR = AR / min(ACE2, RBD)
    
    return fAR

    
def plot_ternary(plot_kin, res, K1, K2, titrated, X1, X2):
    '''
    Plot the fraction of species with respect to the serial dilution of one of them
    --
    plot_kin: whether or not to plot the integrated kinetics as well
    res: number of titrated concentration points that are calculated (affects calculation speed)
    K1: equilibrium dissociation constant between ACE2 and RBD (in M)
    K2: equilibrium dissociation constant between RBD and nAb (in M)
    titrated: the titrated species (x-axis)
    X1: total concentration of fixed species 1 (in M)
    X2: total concentration of fixed speices 2 (in M)
    
    '''         
        
    palette = sns.color_palette('colorblind')
    
    # compute 1D serial dilution
    frac = []
    c_range = np.logspace(-10, -5, res) # [titrated] in the log-domain
    for c in c_range:
        
        if titrated == 'ACE2 (X1=RBD, X2=nAb)':
            
            frac.append(solve_ternary(K1, K2, c, X1, X2))
        
        if titrated == 'RBD (X1=ACE2, X2=nAb)':
            
            frac.append(solve_ternary(K1, K2, X1, c, X2))
        
        if titrated == 'nAb (X1=ACE2, X2=RBD)':
            
            frac.append(solve_ternary(K1, K2, X1, X2, c))

    
    # plot equilibrium results     
    for i, s in enumerate(['ACE2', 'RBD', 'nAb', 'ACE2:RBD', 'RBD:nAb']):
        
        plt.plot(
            c_range, 
            np.array(frac)[:, i, -1], 
            (lambda x: '--' if x != 'ACE2:RBD' else '-')(s),
            label=s, 
            color=palette[i],
            linewidth=(lambda x: 2 if x != 'ACE2:RBD' else 4)(s)
        )
        
    plt.xscale('log')
    plt.xlabel(f'[{titrated.split()[0]}] / M')
    plt.ylabel('Fraction')
    plt.legend(bbox_to_anchor=(1,1))
    plt.show()
    
    # optionally plot the kinetics
    if plot_kin == True:
    
        fig, ax = plt.subplots(ncols=2, figsize=(10, 4))
        
        for i, species in enumerate(['ACE2', 'RBD', 'nAb', 'ACE2:RBD', 'RBD:nAb']):
            
            for sim in range(0, len(c_range)):

                t = np.linspace(0, 10000, 10000)
                a = (sim + 1) / len(c_range) # for gradual transparency

                ax[0].plot(
                    t, 
                    np.array(frac)[sim, i, :], 
                    color=palette[i], 
                    alpha=a
                )

                if a != 1:
                    ax[1].plot(
                        t, 
                        np.gradient(np.array(np.array(frac)[sim, i, :])),
                        color=palette[i],
                        alpha=a
                    )

                else:
                    ax[1].plot(
                        t, 
                        np.gradient(np.array(np.array(frac)[sim, i, :])),
                        color=palette[i],
                        alpha=a,
                        label=species
            )

        ax[0].set(
            xscale='log', 
            xlabel='Integration step',
            ylabel='Fraction',
            title='Time-evolution'
        )
        ax[1].set(
            xscale='log', 
            xlabel='Integration step',
            ylabel='dFraction / dt',
            title='Gradient'
        )
        plt.tight_layout()
        plt.legend(bbox_to_anchor=(1,1))
        plt.show()   


def plot_ternary_2d(res, kace2_rbd, ace2, rbd):
    '''
    Plot phase diagram of ternary sensor activation along [nAb] and K(RBD:nAb) dimensions
    '''
    
    # compute phase-diagram
    ps = []
    c_range = np.logspace(-10, -6, res)
    kd_range = np.logspace(-10, -6, res)
    
    for c in c_range:
        
        for kd in kd_range:
            
            Lmax = solve_ternary_Lmax(kace2_rbd, ace2, rbd) # max signal (without nAb)
            signal = solve_ternary(kace2_rbd, kd, ace2, rbd, c)[3, -1] 
            norm = 1 - (signal / Lmax) # fraction dynamic range lost
            ps.append(norm)
            
    mat = np.array(ps).reshape((res, res))
    
    # replace failed simulations with NaN
    mat[mat > 1.0] = np.nan
    mat[mat < 0.0] = np.nan
    
    # plot
    fig, ax = plt.subplots(figsize=(5, 7))
    cax = ax.imshow(mat, vmin=0, vmax=1, cmap='magma_r', extent=[-10, -6, -6, -10])

    cbar = fig.colorbar(cax, 
                        ticks=[0, 1],
                        label='Fraction of Sensor\nDynamic Range Lost', 
                        shrink=0.5,
                        aspect=10
                       )
    cbar.ax.set_yticklabels([0, 1])
    
    def fake_log(x, pos): # for labelling axes
        return r'$10^{%d}$'%(x)

    ax.xaxis.set_major_formatter(fake_log)
    ax.yaxis.set_major_formatter(fake_log)
    
    ax.invert_xaxis()
    ax.invert_yaxis()
    
    ax.set(xlabel='$K_d$(nAb:RBD)', ylabel='[nAb] / M')
    plt.title(f'$K_d$(ACE2:RBD) = {kace2_rbd:.2e} M\n[ACE2] = {ace2:.2e} M\n[RBD] = {rbd:.2e} M')
    plt.show()


def plot_ternary_2d_bis(res, kace2_rbd, knAb_rbd, nAb):
    '''
    Plot phase diagram of ternary sensor activation along [ACE2] and [RBD] dimensions
    '''
    
    # compute phase-diagram
    ps = []
    c_ace2_range = np.logspace(-10, -7, res)
    c_rbd_range = np.logspace(-9, -6, res)
    
    for cace2 in c_ace2_range:
        
        for crbd in c_rbd_range:
            
            Lmax = solve_ternary_Lmax(kace2_rbd, cace2, crbd) # max signal (without nAb)
            signal = solve_ternary(kace2_rbd, knAb_rbd, cace2, crbd, nAb)[3, -1]
            norm = 1 - (signal / Lmax) # fraction dynamic range lost
            ps.append(norm)
            
    mat = np.array(ps).reshape((res, res))
    
    # replace failed data-points with NaN
    mat[mat > 1.0] = np.nan
    mat[mat < 0.0] = np.nan
    
    # plot
    fig, ax = plt.subplots(figsize=(5, 7))
    cax = ax.imshow(mat, vmin=0, vmax=1, cmap='magma_r', extent=[-9.5, -5.5, -6.5, -10.5])
    
    cbar = fig.colorbar(cax, 
                        ticks=[0, 1],
                        label='Fraction of Sensor\nDynamic Range Lost', 
                        shrink=0.5,
                        aspect=10
                       )
    cbar.ax.set_yticklabels([0, 1])
    
    def fake_log(x, pos): # for labelling axes
        return r'$10^{%d}$'%(x)

    ax.set_xticks(np.log10(c_rbd_range))
    ax.set_yticks(np.log10(c_ace2_range))
    ax.xaxis.set_major_formatter(fake_log)
    ax.yaxis.set_major_formatter(fake_log)
        
    ax.set(xlabel='[RBD] / M', ylabel='[ACE2] / M')
    plt.title(f'$K_d$(ACE2:RBD) = {kace2_rbd:.2e} M\n$K_d$(nAb:RBD) = {knAb_rbd:.2e} M\n[nAb] = {nAb:.2e} M')
    plt.show()



<a id='ternary_system'></a>
# Plotting ternary system

[1D plotting](#tern_1d_plotting)

[2D plotting: K(nAb:RBD) vs. [nAb]](#tern_2d_plotting_nAbvK)

[2D plotting: [RBD] vs. [ACE2]](#tern_2d_plotting_ACE2vRBD)

---

### Interactive plotting for *TERNARY SYSTEM 1D*

- `Plot kinetics`: whether or not to plot the kinetics traces.
- `Resolution`: number of titrated concentration points that are calculated (affects calculation speed).
- `K(ACE2:RBD)`: equilibrium dissociation constant between ACE2 and RBD (in M).
- `K(RBD:nAb)`: equilibrium dissociation constant between RBD and nAb (in M).
- `Titrated`: choice of which species is titrated (x-axis).
- `[X1] / M`: total concentration (in M) of species 1 (not titrated).
- `[X2] / M`: total concentration (in M) of species 2 (not titrated).

The values reported for each species (y-axis) are the fractions of their theortical maximum.

<a id='tern_1d_plotting'></a>

In [3]:
style = {'description_width': 'initial'}

interact(
    plot_ternary,
    plot_kin = widgets.Checkbox(
        value=False,
        description='Plot kinetics'
    ),
    res = widgets.BoundedIntText(
        description='Resolution: ',
        min=10, 
        max=100, 
        value=20
    ),
    K1 = widgets.FloatLogSlider(
        description='K(ACE2:RBD)',
        min=-11, 
        max=-5, 
        step=.1, 
        value=15e-9,
        style=style
    ),
    K2 = widgets.FloatLogSlider(
        description='K(RBD:nAb)',
        min=-11, 
        max=-5, 
        step=.1, 
        value=1e-7
    ),
    titrated = widgets.Dropdown(
        description='Titrated:',
        options=['ACE2 (X1=RBD, X2=nAb)', 'RBD (X1=ACE2, X2=nAb)', 'nAb (X1=ACE2, X2=RBD)'],
        value='nAb (X1=ACE2, X2=RBD)'
    ),
    X1 = widgets.FloatLogSlider(
        description='[X1] / M',
        min=-10, 
        max=-5, 
        step=.1,
        value=1e-7
    ),
    X2 = widgets.FloatLogSlider(
        description='[X2] / M',
        min=-10, 
        max=-5, 
        step=.1, 
        value=1e-7
    )
)

interactive(children=(Checkbox(value=False, description='Plot kinetics'), BoundedIntText(value=20, description…

<function __main__.plot_ternary(plot_kin, res, K1, K2, titrated, X1, X2)>

### Interactive plotting for *TERNARY SYSTEM 2D*: K(nAb:RBD) vs. [nAb]

- `Resolution`: number of "pixels" (affects calculation speed).
- `K(ACE2:RBD)`: equilibrium dissociation constant between ACE2 and RBD (in M).
- `[ACE2] / M`: total concentration of ACE2 (in M).
- `[RBD] / M`: total concentration of RBD (in M).

The `Fraction of Sensor Dynamic Range Lost` is defined as `1 - (signal / max)`, where `signal` is the fraction of ACE2:RBD complex (signalling-competent complex), and `max` is the signal observed under the same conditions but *without* the presence of nAb.

<a id='tern_2d_plotting_nAbvK'></a>

In [4]:
# interactive plotting    
style = {'description_width': 'initial'}
interact(
    plot_ternary_2d,
    res = widgets.BoundedIntText(
        description='Resolution: ',
        min=5,
        max=50,
        value=10
    ),
    kace2_rbd = widgets.FloatLogSlider(
        description='K(ACE2:RBD)',
        min=-10,
        max=-5,
        value=15e-9,
        style=style
    ),
    ace2 = widgets.FloatLogSlider(
        description='[ACE2] / M',
        min=-12,
        max=-5,
        value=10e-9
    ),
    rbd = widgets.FloatLogSlider(
        description='[RBD] / M',
        min=-10,
        max=-5,
        value=10e-9
    )
)

interactive(children=(BoundedIntText(value=10, description='Resolution: ', max=50, min=5), FloatLogSlider(valu…

<function __main__.plot_ternary_2d(res, kace2_rbd, ace2, rbd)>

### Interactive plotting for *TERNARY SYSTEM 2D*: [RBD] vs. [ACE2]

- `Resolution`: number of "pixels" (affects calculation speed).
- `K(ACE2:RBD)`: equilibrium dissociation constant between ACE2 and RBD (in M).
- `K(nAb:RBD)`: equilibrium dissociation constant between nAb and RBD (in M).
- `[nAb] / M`: total concentration of nAb (in M).

The `Fraction of Sensor Dynamic Range Lost` is defined as `1 - (signal / max)`, where `signal` is the fraction of ACE2:RBD complex (signalling-competent complex), and `max` is the signal observed under the same conditions but *without* the presence of nAb.

<a id='tern_2d_plotting_ACE2vRBD'></a>

In [5]:
# interactive plotting    
style = {'description_width': 'initial'}
interact(
    plot_ternary_2d_bis,
    res = widgets.BoundedIntText(
        description='Resolution: ',
        min=2,
        max=50,
        value=4
    ),
    kace2_rbd = widgets.FloatLogSlider(
        description='K(ACE2:RBD)',
        min=-10,
        max=-5,
        value=15e-9,
        style=style
    ),
    knAb_rbd = widgets.FloatLogSlider(
        description='K(nAb:RBD)',
        min=-10,
        max=-5,
        value=15e-9,
        style=style
    ),
    nAb = widgets.FloatLogSlider(
        description='[nAb] / M',
        min=-10,
        max=-5,
        value=10e-9
    )
)

interactive(children=(BoundedIntText(value=4, description='Resolution: ', max=50, min=2), FloatLogSlider(value…

<function __main__.plot_ternary_2d_bis(res, kace2_rbd, knAb_rbd, nAb)>

# Quaternary system
---

>__Species:__\
Cage-Latch\
Key\
RBD\
nAb\
\
__Equilibria:__\
Cage-Latch (closed) <=> Cage-Latch (open)\
Cage-Latch (open) + Key <=> Key:Cage-Latch\
Cage-Latch (open) + RBD <=> Cage-Latch:RBD\
Key:Cage-Latch + RBD <=> Key:Cage-Latch:RBD\
Cage-Latch:RBD + Key <=> Key:Cage-Latch:RBD\
RBD + nAb <=> RBD:nAb

- The Cage-Latch can exist in an open (binding-competent) or closed (binding-incompetent) state.

- The RBD minibinder (LCB1) is contained within the Latch.

- LCB1 (contained within the Cage-Latch) and RBD can interact (in the open state).

- The Key can interact with the Cage-Latch (in the open state).

- RBD and nAb can interact.

- Both the binary complex Key:Cage-Latch and the ternary complex Cage-Latch:Key:RBD give out the signal.

In [6]:
def ODEsystem_quaternary(conc, t, ks): # system of ODEs to be solved by odeint()
    '''
    Returns a system of ODEs for numerical integration
    Describes system when all components are present (Cage-Latch, Key, RBD, nAb)
    ---
    t: time-step of the ODE integration
    Conc: tuple of concentrations; unknowns
    k: tuple of kinetic constants; constants of the system
    ---
    C = closed cage (binding-incompetent)
    O = open cage (binding-competent)
    A = Key
    B = RBD
    X = nAb
    Affinity of the latch to the cage is defined by K = k_ / k
    Affinity of A:O complex is defined by K1 = k_1 / k1 
    Affinity of O:B complex is defined by K2 = k_2 / k2
    Affinity of B:X complex is defined by K3 = k_3 / k3
    '''
    
    C, O, A, B, X, AO, OB, AOB, BX = conc    
    
    # underscores indicate dissociation rate constants
    # others are association rate constants
    k, k_, k1, k_1, k2, k_2, k3, k_3 = ks

    # coupled ODEs describing the system
    dC_dt   = - k*C + k_*O
    dO_dt   = k*C - k_*O - k1*A*O + k_1*AO - k2*O*B + k_2*OB
    dA_dt   = - k1*A*O + k_1*AO - k1*A*OB + k_1*AOB
    dB_dt   = - k2*O*B + k_2*OB - k2*AO*B + k_2*AOB - k3*B*X + k_3*BX
    dX_dt   = - k3*B*X + k_3*BX
    dAO_dt  = k1*A*O - k_1*AO - k2*AO*B + k_2*AOB 
    dOB_dt  = k2*O*B - k_2*OB - k1*A*OB + k_1*AOB
    dAOB_dt = k1*A*OB - k_1*AOB + k2*AO*B - k_2*AOB
    dBX_dt  = k3*B*X - k_3*BX

    return [
        dC_dt, 
        dO_dt, 
        dA_dt, 
        dB_dt,
        dX_dt,
        dAO_dt, 
        dOB_dt, 
        dAOB_dt,
        dBX_dt
    ]

def ODEsystem_quaternary_Lmax(conc, t, ks): # system of ODEs to be solved by odeint()
    '''
    Returns a system of ODEs for numerical integration
    Describes system when all but nAb is present (Cage-Latch, Key, RBD)
    For getting 'max' sensor signal without nAb addition
    ---
    t: time-step of the ODE integration
    Conc: tuple of concentrations; unknowns
    k: tuple of kinetic constants; constants of the system
    ---
    C = closed cage (binding-incompetent)
    O = open cage (binding-compotent)
    A = Key
    B = RBD
    Affinity of the latch to the cage is defined by K = k_ / k
    Affinity of A:O complex is defined by K1 = k_1 / k1 
    Affinity of O:B complex is defined by K2 = k_2 / k2
    '''
    
    C, O, A, B, AO, OB, AOB = conc    
    
    # underscores indicate dissociation rate constants
    # others are association rate constants
    k, k_, k1, k_1, k2, k_2 = ks

    # coupled ODEs describing the system
    dC_dt   = - k*C + k_*O
    dO_dt   = k*C - k_*O - k1*A*O + k_1*AO - k2*O*B + k_2*OB
    dA_dt   = - k1*A*O + k_1*AO - k1*A*OB + k_1*AOB
    dB_dt   = - k2*O*B + k_2*OB - k2*AO*B + k_2*AOB
    dAO_dt  = k1*A*O - k_1*AO - k2*AO*B + k_2*AOB 
    dOB_dt  = k2*O*B - k_2*OB - k1*A*OB + k_1*AOB
    dAOB_dt = k1*A*OB - k_1*AOB + k2*AO*B - k_2*AOB

    return [
        dC_dt, 
        dO_dt, 
        dA_dt, 
        dB_dt,
        dAO_dt, 
        dOB_dt, 
        dAOB_dt,
    ]


def ODEsystem_quaternary_Lmin(conc, t, ks): # system of ODEs to be solved by odeint()
    '''
    Returns a system of ODEs for numerical integration
    Describes system when only Cage-Latch and Key are present (no RBD, no nAb)
    For getting 'min' sensor signal ('leaky signal')
    ---
    t: time-step of the ODE integration
    Conc: tuple of concentrations; unknowns
    k: tuple of kinetic constants; constants of the system
    ---
    C = closed cage (binding-incompetent)
    O = open cage (binding-compotent)
    A = Key
    Affinity of the latch to the cage is defined by K = k_ / k
    Affinity of A:O complex is defined by K1 = k_1 / k1 
    '''
    
    C, O, A, AO = conc    
    
    # underscores indicate dissociation rate constants
    # others are association rate constants
    k, k_, k1, k_1 = ks

    # coupled ODEs describing the system
    dC_dt   = - k*C + k_*O
    dO_dt   = k*C - k_*O - k1*A*O + k_1*AO
    dA_dt   = - k1*A*O + k_1*AO
    dAO_dt  = k1*A*O - k_1*AO 

    return [
        dC_dt, 
        dO_dt, 
        dA_dt, 
        dAO_dt, 
    ]

def solve_quaternary(dGopen, K1, K2, K3, Cage, Key, RBD, nAb):
    '''
    Solves the equililibrum of the system as a function of affinities and concentrations
    Returns the the time-evolution of the fraction of each species
    --
    dGopen: the free energy for separating the Latch from the Cage
    K1: equilibrium dissociation constant between Key and Cage-Latch (in M)
    K2: equilibrium dissociation constant between RBD and Cage-Latch (in M)
    K3: equilibrium dissociation constant between RBD and nAb (in M)
    Cage: total concentration of Cage (in M)
    Key: total concentration of Key (in M)
    RBD: total concentration of RBD (in M)
    nAb: total concentration of nAb (in M)
    '''

    # convert free energy to an equilibirum constant
    # units of dGopen are assumed to be in kcal/mol, and T = 25C
    K = np.exp( dGopen / (1.9858775e-3 * 298.15)) 
    
    k, k1, k2, k3 = 1e-3, 1e6, 1e6, 1e6 # arbitrary association rate constants (in M^-1 s^-1)
    # although the model is based on ODEs (kinetics), only consider steady-state
    # thus the actual kinetic rates are unimportant, only the ratios (equilibrium) matter
    
    # get dissociation rate constants from association rate constants and equilibrium dissociation constants
    k_ = k * K
    k_1 = k1 * K1
    k_2 = k2 * K2
    k_3 = k3 * K3
    
    ks = k, k_, k1, k_1, k2, k_2, k3, k_3
 
    # inital concentrations used by ODE solver (in M)
    C0 = 0
    O0 = Cage # to speed-up equilibration?
    A0 = Key
    B0 = RBD 
    X0 = nAb
    AO0 = 0
    OB0 = 0
    AOB0 = 0
    BX0 = 0    
   
    frac = []

    init = [C0, O0, A0, B0, X0, AO0, OB0, AOB0, BX0]

    # perform integration over 1s first, then increase if system has not reach equilibrium
    sections = 0
    maxcounter = 0
    reached_eq = [False] * len(init)

    while not all(reached_eq) and maxcounter < 50: # adaptive integration until steady-state is reached

        t = np.linspace(0, 10**sections, 10000)

        # solve the system of ODEs
        sol = odeint(
            ODEsystem_quaternary, 
            init, 
            t,
            args=(ks,),
            mxstep=5000
        )

        # compute fraction for each species
        frac.append(sol[:, 0] / Cage)
        frac.append(sol[:, 1] / Cage)
        frac.append(sol[:, 2] / Key)
        frac.append(sol[:, 3] / RBD)
        frac.append(sol[:, 4] / nAb)          
        frac.append(sol[:, 5] / min(Key, Cage))            
        frac.append(sol[:, 6] / min(Cage, RBD))            
        frac.append(sol[:, 7] / min(Key, Cage, RBD))            
        frac.append(sol[:, 8] / min(RBD, nAb))            
        frac.append((sol[:, 5] / min(Key, Cage)) + (sol[:, 7] / min(Key, Cage, RBD)))

        # test if system reached equilibrium by computing the gradient
        gradients = np.array(np.gradient(frac, axis=-1))
        reached_eq = [True if abs(endpoint) < 1e-8 else False for endpoint in np.mean(gradients[:, -10:], axis=-1)]

        if not all(reached_eq) and maxcounter < 50: # check if system reached equilibrium
            frac = [] # erase
            sections += 1  
        
        maxcounter += 1
    
    if np.array(frac).shape != (10, 10000):
        print(f'Simulation did not converge for: dGopen = {dGopen:.1e}, K1 = {K1:.1e}, K2 = {K2:.1e}, K3 = {K3:.1e}, \
        [Cage] = {Cage:.1e}, [Key] = {Key:.1e}, [RBD] = {RBD:.1e}, [nAb] = {nAb:.1e}')
        frac = np.ones((10, 10000))

    return np.array(frac)

def solve_quaternary_Lmax(dGopen, K1, K2, Cage, Key, RBD):
    '''
    Solves the equililibrum of the system as a function of affinities and concentrations
    Returns the the time-evolution of the fraction of each species
    --
    dGopen: the free energy for separating the Latch from the Cage
    K1: equilibrium dissociation constant between Key and Cage-Latch (in M)
    K2: equilibrium dissociation constant between RBD and Cage-Latch (in M)
    Cage: total concentration of Cage (in M)
    Key: total concentration of Key (in M)
    RBD: total concentration of RBD (in M)
    '''

    # convert free energy to an equilibirum constant
    # units of dGopen are assumed to be in kcal/mol, and T = 25C
    K = np.exp( dGopen / (1.9858775e-3 * 298.15)) 
    
    k, k1, k2 = 1e-3, 1e6, 1e6 # arbitrary association rate constants (in M^-1 s^-1)
    # although the model is based on ODEs (kinetics), only consider steady-state
    # thus the actual kinetic rates are unimportant, only the ratios (equilibrium) matter
    
    # get dissociation rate constants from association rate constants and equilibrium dissociation constants
    k_ = k * K
    k_1 = k1 * K1
    k_2 = k2 * K2
    
    ks = k, k_, k1, k_1, k2, k_2
 
    # inital concentrations used by ODE solver (in M)
    C0 = 0
    O0 = Cage # to speed-up equilibration?
    A0 = Key
    B0 = RBD 
    AO0 = 0
    OB0 = 0
    AOB0 = 0 
   
    frac = []

    init = [C0, O0, A0, B0, AO0, OB0, AOB0]

    # perform integration over 1s first, then increase if system has not reach equilibrium
    sections = 0
    maxcounter = 0
    reached_eq = [False] * len(init)

    while not all(reached_eq) and maxcounter < 50: # adaptive integration until steady-state is reached

        t = np.linspace(0, 10**sections, 10000)

        # solve the system of ODEs
        sol = odeint(
            ODEsystem_quaternary_Lmax, 
            init, 
            t,
            args=(ks,),
            mxstep=5000
        )

        # compute fraction for each species
        frac.append(sol[:, 0] / Cage)
        frac.append(sol[:, 1] / Cage)
        frac.append(sol[:, 2] / Key)
        frac.append(sol[:, 3] / RBD)
        frac.append(sol[:, 4] / min(Key, Cage))            
        frac.append(sol[:, 5] / min(Cage, RBD))            
        frac.append(sol[:, 6] / min(Key, Cage, RBD))            
        frac.append((sol[:, 4] / min(Key, Cage)) + (sol[:, 6] / min(Key, Cage, RBD)))

        # test if system reached equilibrium by computing the gradient
        gradients = np.array(np.gradient(frac, axis=-1))
        reached_eq = [True if abs(endpoint) < 1e-8 else False for endpoint in np.mean(gradients[:, -10:], axis=-1)]

        if not all(reached_eq) and maxcounter < 50: # check if system reached equilibrium
            frac = [] # erase
            sections += 1  
        
        maxcounter += 1
    
    if np.array(frac).shape != (8, 10000):
        print(f'Simulation did not converge for Lmax: dGopen = {dGopen:.1e}, K1 = {K1:.1e}, K2 = {K2:.1e}, \
        [Cage] = {Cage:.1e}, [Key] = {Key:.1e}, [RBD] = {RBD:.1e}')
        frac = np.ones((8, 10000))

    return np.array(frac)


def solve_quaternary_Lmin(dGopen, K1, Cage, Key):
    '''
    Solves the equililibrum of the system as a function of affinities and concentrations
    Returns the the time-evolution of the fraction of each species
    --
    dGopen: the free energy for separating the Latch from the Cage
    K1: equilibrium dissociation constant between Key and Cage-Latch (in M)
    Cage: total concentration of Cage (in M)
    Key: total concentration of Key (in M)
    '''

    # convert free energy to an equilibirum constant
    # units of dGopen are assumed to be in kcal/mol, and T = 25C
    K = np.exp( dGopen / (1.9858775e-3 * 298.15)) 

    k, k1 = 1e-3, 1e6 # arbitrary association rate constants (in M^-1 s^-1)
    # although the model is based on ODEs (kinetics), only consider steady-state
    # thus the actual kinetic rates are unimportant, only the ratios (equilibrium) matter
    
    # get dissociation rate constants from association rate constants and equilibrium dissociation constants
    k_ = k * K
    k_1 = k1 * K1
    
    ks = k, k_, k1, k_1
 
    # inital concentrations used by ODE solver (in M)
    C0 = 0
    O0 = Cage # to speed-up equilibration?
    A0 = Key
    AO0 = 0
   
    frac = []

    init = [C0, O0, A0, AO0]

    # perform integration over 1s first, then increase if system has not reach equilibrium
    sections = 0
    maxcounter = 0
    reached_eq = [False] * len(init)

    while not all(reached_eq) and maxcounter < 50: # adaptive integration until steady-state is reached

        t = np.linspace(0, 10**sections, 10000)

        # solve the system of ODEs
        sol = odeint(
            ODEsystem_quaternary_Lmin,
            init, 
            t,
            args=(ks,),
            mxstep=5000
        )

        # compute fraction for each species
        frac.append(sol[:, 0] / Cage)
        frac.append(sol[:, 1] / Cage)
        frac.append(sol[:, 2] / Key)
        frac.append(sol[:, 3] / min(Key, Cage))                                 

        # test if system reached equilibrium by computing the gradient
        gradients = np.array(np.gradient(frac, axis=-1))
        reached_eq = [True if abs(endpoint) < 1e-8 else False for endpoint in np.mean(gradients[:, -10:], axis=-1)]

        if not all(reached_eq) and maxcounter < 50: # check if system reached equilibrium
            frac = [] # erase
            sections += 1  
        
        maxcounter += 1
    
    if np.array(frac).shape != (4, 10000):
        print(f'Simulation did not converge for Lmin: dGopen = {dGopen:.1e}, K1 = {K1:.1e}, [Cage] = {Cage:.1e}, [Key] = {Key:.1e}')
        frac = np.ones((4, 10000))

    return np.array(frac)


def plot_quaternary(plot_kin, res, dGopen, K1, K2, K3, titrated, X1, X2, X3):
    '''
    Plot the fraction of species with respect to the serial dilution of one of them
    --
    plot_kin: whether or not to plot the kinetics as well (Boolean)
    res: number of titrated concentration points that are calculated (affects calculation speed)
    dGopen: the free energy for separating the Latch from the Cage
    K1: equilibrium dissociation constant between Key and Cage (in M)
    K2: equilibrium dissociation constant between RBD and Cage (in M)
    K3: equilibrium dissociation constant between RBD and nAb (in M)
    titrated: the titrated species (x-axis)
    X1: total concentration of species 1 (in M)
    X2: total concentration of species 2 (in M)
    X3: total concentration of species 3 (in M)
    
    '''
    
    palette = sns.color_palette('colorblind')
    
    # compute 1D serial dilution
    frac = []
    c_range = np.logspace(-10, -5, res) # [titrated] in the log-domain
    for c in c_range:

        if titrated == 'Cage (X1=Key, X2=RBD, X3=nAb)':
            
            frac.append(solve_quaternary(dGopen, K1, K2, K3, c, X1, X2, X3))
        
        if titrated == 'Key (X1=Cage, X2=RBD, X3=nAb)':
            
            frac.append(solve_quaternary(dGopen, K1, K2, K3, X1, c, X2, X3))
        
        if titrated == 'RBD (X1=Cage, X2=Key, X3=nAb)':
            
            frac.append(solve_quaternary(dGopen, K1, K2, K3, X1, X2, c, X3))
        
        if titrated == 'nAb (X1=Cage, X2=Key, X3=RBD)':
            
            frac.append(solve_quaternary(dGopen, K1, K2, K3, X1, X2, X3, c))
            
    
    # plot equilibrium results     
    for i, s in enumerate([
        'Cage-Latch (closed)', 
        'Cage-Latch (open)', 
        'Key', 
        'RBD',
        'nAb',
        'Key:Cage-Latch',
        'Cage-Latch:RBD',
        'Key:Cage-Latch:RBD',
        'RBD:nAb',
        'Tot. Signal'
    ]):
        

        plt.plot(
            c_range, 
            np.array(frac)[:, i, -1], 
            (lambda x: '--' if x !='Tot. Signal' else '-')(s),
            label=s, 
            color=palette[i],
            linewidth=(lambda x: 2 if x !='Tot. Signal' else 4)(s)
        )   
    
    plt.xscale('log')
    plt.xlabel(f'[{titrated.split()[0]}] / M')
    plt.ylabel('Fraction')
    plt.legend(bbox_to_anchor=(1,1))
    plt.show()    
    
    
    # optionally plot the kinetics
    if plot_kin == True:
    
        fig, ax = plt.subplots(ncols=2, figsize=(10,4))
        
        for i, species in enumerate([
            'Cage-Latch (closed)', 
            'Cage-Latch (open)', 
            'Key', 
            'RBD',
            'nAb',
            'Key:Cage-Latch',
            'Cage-Latch:RBD',
            'Key:Cage-Latch:RBD',
            'RBD:nAb'

        ]):
            
            for sim in range(0, len(c_range)):

                t = np.linspace(0, 10000, 10000)
                a = (sim + 1) / len(c_range)

                ax[0].plot(
                    t, 
                    np.array(frac)[sim, i, :], 
                    color=palette[i], 
                    alpha=a
                )

                if a != 1:
                    ax[1].plot(
                        t, 
                        np.gradient(np.array(np.array(frac)[sim, i, :])),
                        color=palette[i],
                        alpha=a
                    )

                else:
                    ax[1].plot(
                        t, 
                        np.gradient(np.array(np.array(frac)[sim, i, :])),
                        color=palette[i],
                        alpha=a,
                        label=species
            )

        ax[0].set(
            xscale='log', 
            xlabel='Integration step',
            ylabel='Fraction',
            title='Time-evolution'
        )
        ax[1].set(
            xscale='log', 
            xlabel='Integration step',
            ylabel='dFraction / dt',
            title='Gradient'
        )
        plt.tight_layout()
        plt.legend(bbox_to_anchor=(1,1))
        plt.show()   
        

def plot_quaternary_2d(res, dGopen, K1, K2, cage, key, rbd):
    '''
    Plot phase diagram of quaternary sensor activation along [nAb] and K(RBD:nAb) dimensions
    '''
    
    # compute phase-diagram
    ps = []
    c_range = np.logspace(-10, -6, res)
    kd_range = np.logspace(-10, -6, res)
    
    for c in c_range:
        
        for kd in kd_range:
            
            Lmin = solve_quaternary_Lmin(dGopen, K1, cage, key)[-1, -1]
            Lmax = solve_quaternary_Lmax(dGopen, K1, K2, cage, key, rbd)[-1, -1]
            signal = solve_quaternary(dGopen, K1, K2, kd, cage, key, rbd, c)[-1, -1]
            norm = 1 - ((signal - Lmin) / (Lmax - Lmin))
            
            ps.append(norm)
    
    mat = np.array(ps).reshape((res, res))
    
    # replace failed data-points with NaN
    mat[mat > 1.0] = np.nan
    mat[mat < 0.0] = np.nan
    
    # plot
    fig, ax = plt.subplots(figsize=(5, 7))
    cax = ax.imshow(mat, vmin=0, vmax=1, cmap='magma_r', extent=[-10, -6, -6, -10])
    
    cbar = fig.colorbar(cax, 
                        ticks=[0, 1],
                        label='Fraction of lucCageRBD\nDynamic Range Lost', 
                        shrink=0.5,
                        aspect=10
                       )
    cbar.ax.set_yticklabels([0, 1])
    
    def fake_log(x, pos): # for labelling axes
        return r'$10^{%d}$'%(x)

    ax.xaxis.set_major_formatter(fake_log)
    ax.yaxis.set_major_formatter(fake_log)
    
    ax.invert_xaxis()
    ax.invert_yaxis()
    
    ax.set(xlabel='$K_d$(nAb:RBD)', ylabel='[nAb] / M')
    plt.title(
        f'$\Delta G$open = {dGopen:.1f} kcal/mol\n\
        K(Key:Cage) = {K1:.2e} M\n\
        K(Latch:RBD) = {K2:.2e} M\n\
        [Cage-Latch] = {cage:.2e} M\n\
        [Key] = {key:.2e} M\n\
        [RBD] = {rbd:.2e} M'
    )

    plt.show()

    
def plot_quaternary_2d_bis(res, dGopen, K1, K2, K3, cage, nAb):
    '''
    Plot phase diagram of quaternary sensor activation along [Key] and [RBD] dimensions
    '''
    
    # compute phase-diagram
    ps = []
    c_key_range = np.logspace(-10, -7, res)
    c_rbd_range = np.logspace(-9, -6, res)

    for ckey in c_key_range:
        
        for crbd in c_rbd_range:
            
            Lmin = solve_quaternary_Lmin(dGopen, K1, cage, ckey)[-1, -1]
            Lmax = solve_quaternary_Lmax(dGopen, K1, K2, cage, ckey, crbd)[-1, -1]
            signal = solve_quaternary(dGopen, K1, K2, K3, cage, ckey, crbd, nAb)[-1, -1]
            norm = 1 - ((signal - Lmin) / (Lmax - Lmin))
            
            ps.append(norm)
    
    mat = np.array(ps).reshape((res, res))
    
    # replace failed data-points with NaN
    mat[mat > 1] = np.nan
    mat[mat < 0] = np.nan
    
    # plot
    fig, ax = plt.subplots(figsize=(5, 7))
    cax = ax.imshow(mat, vmin=0, vmax=1, cmap='magma_r', extent=[-9.5, -5.5, -6.5, -10.5])
    
    cbar = fig.colorbar(cax, 
                        ticks=[0, 1],
                        label='Fraction of lucCageRBD\nDynamic Range Lost', 
                        shrink=0.5,
                        aspect=10
                       )
    cbar.ax.set_yticklabels([0, 1])
    
    def fake_log(x, pos): # for labelling axes
        return r'$10^{%d}$'%(x)

    ax.set_xticks(np.log10(c_rbd_range))
    ax.set_yticks(np.log10(c_key_range))
    ax.xaxis.set_major_formatter(fake_log)
    ax.yaxis.set_major_formatter(fake_log)
        
    ax.set(xlabel='[RBD] / M', ylabel='[Key] / M')
    plt.title(
        f'$\Delta G$open = {dGopen:.1f} kcal/mol\n\
        K(Key:Cage) = {K1:.2e} M\n\
        K(Latch:RBD) = {K2:.2e} M\n\
        K(RBD:nAb) = {K3:.2e} M\n\
        [Cage-Latch] = {cage:.2e} M\n\
        [nAb] = {nAb:.2e} M'
    )


<a id='quaternary_system'></a>
# Plotting quaternary system

[1D plotting](#quat_1d_plotting)

[2D plotting: K(nAb:RBD) vs. [nAb]](#quat_2d_plotting_nAbvK)

[2D plotting: [RBD] vs. [Key]](#quat_2d_plotting_KeyvRBD)

---

### Interactive plotting for *QUATERNARY SYSTEM 1D*

- `Plot kinetics`: whether or not to plot the kinetics traces.
- `Resolution`: number of titrated concentration points that are calculated (affects calculation speed).
- $\Delta G$(open): the free energy of Cage-Latch opening.
- `K(Key:Cage)`: equilibrium dissociation constant between Key and open Cage-Latch (in M).
- `K(Latch:RBD)`: equilibrium dissociation constant between RBD and open Cage-Latch (in M).
- `K(RBD:nAb)`: equilibrium dissociation constant between RBD and nAb (in M).
- `Titrated`: choice of which species is titrated (x-axis).
- `[X1] / M`: total concentration (in M) of species 1 (not titrated).
- `[X2] / M`: total concentration (in M) of species 2 (not titrated).
- `[X3] / M`: total concentration (in M) of species 3 (not titrated).

The values reported for each species (y-axis) are the fractions of their theortical maximum.

The total signal is the sum of the fractions of Key:Cage-Latch and Key:Cage-Latch:RBD

<a id='quat_1d_plotting'></a>

In [11]:
style = {'description_width': 'initial'}

interact(
    plot_quaternary,
    plot_kin = widgets.Checkbox(
        value=False,
        description='Plot kinetics'
    ),
    res = widgets.BoundedIntText(
        description='Resolution: ',
        min=10, 
        max=100, 
        value=20
    ),
    dGopen = widgets.FloatSlider(
        description='$\Delta G$(open)',
        min=0,
        max=10,
        value=4
    ),
    K1 = widgets.FloatLogSlider(
        description='K(Key:Cage)',
        min=-12, 
        max=-5, 
        step=.1, 
        value=5e-9,
        style=style
    ),
    K2 = widgets.FloatLogSlider(
        description='K(Latch:RBD)',
        min=-12, 
        max=-5, 
        step=.1, 
        value=5e-10,
        style=style
    ),
    K3 = widgets.FloatLogSlider(
        description='K(RBD:nAb)',
        min=-12, 
        max=-5, 
        step=.1,
        value=1e-7,
        style=style
    ),
    titrated = widgets.Dropdown(
        description='Titrated: ',
        options=[
            'Cage (X1=Key, X2=RBD, X3=nAb)',
            'Key (X1=Cage, X2=RBD, X3=nAb)',
            'RBD (X1=Cage, X2=Key, X3=nAb)',
            'nAb (X1=Cage, X2=Key, X3=RBD)',
        ],
        value='nAb (X1=Cage, X2=Key, X3=RBD)',
        style=style
    ),
    X1 = widgets.FloatLogSlider(
        description='[X1] / M',
        min=-10, 
        max=-4, 
        step=.1, 
        value=1e-9, 
    ),
    X2 = widgets.FloatLogSlider(
        description='[X2] / M',
        min=-10, 
        max=-4, 
        step=.1, 
        value=5e-9
    ),
    X3 = widgets.FloatLogSlider(
        description='[X3] / M',
        min=-10, 
        max=-4, 
        step=.1, 
        value=5e-9
    )
)

interactive(children=(Checkbox(value=False, description='Plot kinetics'), BoundedIntText(value=20, description…

<function __main__.plot_quaternary(plot_kin, res, dGopen, K1, K2, K3, titrated, X1, X2, X3)>

### Interactive plotting for *QUATERNARY SYSTEM 2D*: [nAb] vs. K(nAb:RBD)

- `Resolution`: number of "pixels" (affects calculation speed).
- $\Delta G$(open): the free energy of Cage-Latch opening.
- `K(Key:Cage)`: equilibrium dissociation constant between Key and open Cage-Latch (in M).
- `K(Latch:RBD)`: equilibrium dissociation constant between RBD and open Cage-Latch (in M).
- `[Cage-Latch] / M`: total concentration of Cage-Latch (in M).
- `[Key] / M`: total concentration of Key (in M).
- `[RBD] / M`: total concentration of RBD (in M).

The `Fraction of lucCageRBD Dynamic Range Lost` is defined as `1 - ((signal - min) / (max - min))`, where `signal` is sum of the fractions of `Key:Cage-Latch` and `Key:Cage-Latch:RBD` (the signalling-competent complexes), `max` is the signal observed under the same conditions but *without* the presence of nAb, and `min` is the signal observed when only `Key` and `Cage-Latch` are present.

<a id='quat_2d_plotting_nAbvK'></a>

In [8]:
style = {'description_width': 'initial'}

interact(
    plot_quaternary_2d,
    res = widgets.BoundedIntText(
        description='Resolution: ',
        min=5,
        max=50,
        value=10
    ),
    dGopen = widgets.FloatSlider(
        description='$\Delta G$(open)',
        min=0,
        max=15,
        value=4
    ),
    K1 = widgets.FloatLogSlider(
        description='K(Key:Cage)',
        min=-12, 
        max=-5, 
        step=.1, 
        value=5e-9,
        style=style
    ),
    K2 = widgets.FloatLogSlider(
        description='K(Latch:RBD)',
        min=-12, 
        max=-5, 
        step=.1, 
        value=5e-10,
        style=style
    ),
    cage = widgets.FloatLogSlider(
        description='[Cage-Latch] / M',
        min=-10, 
        max=-4, 
        step=.1, 
        value=1e-9, 
        style=style
    ),
    key = widgets.FloatLogSlider(
        description='[Key] / M',
        min=-10, 
        max=-4, 
        step=.1, 
        value=1e-7
    ),
    rbd = widgets.FloatLogSlider(
        description='[RBD] / M',
        min=-10, 
        max=-4, 
        step=.1, 
        value=1e-7
    )
)

interactive(children=(BoundedIntText(value=10, description='Resolution: ', max=50, min=5), FloatSlider(value=2…

<function __main__.plot_quaternary_2d(res, dGopen, K1, K2, cage, key, rbd)>

### Interactive plotting for *QUATERNARY SYSTEM 2D*: [Key] vs. [RBD]

- `Resolution`: number of "pixels" (affects calculation speed).
- $\Delta G$(open): the free energy of Cage-Latch opening.
- `K(Key:Cage)`: equilibrium dissociation constant between Key and open Cage-Latch (in M).
- `K(Latch:RBD)`: equilibrium dissociation constant between RBD and open Cage-Latch (in M).
- `K(nAb:RBD)`: equilibrium dissociation constant between nAb and RBD (in M).
- `[Cage-Latch] / M`: total concentration of Cage-Latch (in M).
- `[nAb] / M`: total concentration of nAb (in M).

The `Fraction of lucCageRBD Dynamic Range Lost` is defined as `1 - ((signal - min) / (max - min))`, where `signal` is sum of the fractions of `Key:Cage-Latch` and `Key:Cage-Latch:RBD` (the signalling-competent complexes), `max` is the signal observed under the same conditions but *without* the presence of nAb, and `min` is the signal observed when only `Key` and `Cage-Latch` are present.

<a id='quat_2d_plotting_KeyvRBD'></a>

In [10]:
style = {'description_width': 'initial'}

interact(
    plot_quaternary_2d_bis,
    res = widgets.BoundedIntText(
        description='Resolution: ',
        min=2,
        max=50,
        value=4
    ),
    dGopen = widgets.FloatSlider(
        description='$\Delta G$(open)',
        min=0,
        max=15,
        value=4
    ),
    K1 = widgets.FloatLogSlider(
        description='K(Key:Cage)',
        min=-12, 
        max=-5, 
        step=.1, 
        value=5e-9,
        style=style
    ),
    K2 = widgets.FloatLogSlider(
        description='K(Latch:RBD)',
        min=-12, 
        max=-5, 
        step=.1, 
        value=5e-10,
        style=style
    ),
    K3 = widgets.FloatLogSlider(
        description='K(RBD:nAb)',
        min=-12, 
        max=-5, 
        step=.1, 
        value=1e-8,
        style=style
    ),
    cage = widgets.FloatLogSlider(
        description='[Cage-Latch] / M',
        min=-10, 
        max=-4, 
        step=.1, 
        value=1e-9,
        style=style
    ),
    nAb = widgets.FloatLogSlider(
        description='[nAb] / M',
        min=-10, 
        max=-4, 
        step=.1, 
        value=100e-9
    )
)

interactive(children=(BoundedIntText(value=4, description='Resolution: ', max=50, min=2), FloatSlider(value=4.…

<function __main__.plot_quaternary_2d_bis(res, dGopen, K1, K2, K3, cage, nAb)>